# Create Meteorite Shapefiles

In [228]:
import os
import pandas as pd
import geopandas as gpd
import shapely

from zipfile import ZipFile

### Create a data frame

In [229]:
# Create a data frame from a CSV file
df = pd.read_csv("./assets/meteorite-data-nasa.csv")

### Wrangle the data

In [230]:
# Get rid of unnecessary columns
keep_cols = [
    "name",
    "mass (g)",
    "fall",
    "year",
    "reclat",
    "reclong"
]
df = df[keep_cols]

In [231]:
# Show percentage of rows that have null reclong or reclat values
(df["reclong"].isna() | df["reclat"].isna()).mean()

np.float64(0.16000962463907603)

In [232]:
# Keep only those rows where reclong nor reclat is null
df = df[(df["reclong"].notna() & df["reclat"].notna())]

In [233]:
# Now the null percentage is 0
(df["reclong"].isna() | df["reclat"].isna()).mean()

np.float64(0.0)

### Create a geo data frame

In [234]:
# Construct a geo data frame
geometry = shapely.points(df.pop("reclong"), df.pop("reclat"))
gdf = gpd.GeoDataFrame(df, geometry = geometry)

In [235]:
# Set its Coordinate Reference System
gdf.set_crs("EPSG:4326")

,name,mass (g),fall,year,geometry
0,Aachen,21.0,Fell,1880.0,POINT (6.08333 50.775)
1,Aarhus,720.0,Fell,1951.0,POINT (10.23333 56.18333)
2,Abee,107000.0,Fell,1952.0,POINT (-113 54.21667)
3,Acapulco,1914.0,Fell,1976.0,POINT (-99.9 16.88333)
4,Achiras,780.0,Fell,1902.0,POINT (-64.95 -33.16667)
...,...,...,...,...,...
45711,Zillah 002,172.0,Found,1990.0,POINT (17.0185 29.037)
45712,Zinder,46.0,Found,1999.0,POINT (8.96667 13.78333)
45713,Zlin,3.3,Found,1939.0,POINT (17.66667 49.25)
45714,Zubkovsky,2167.0,Found,2003.0,POINT (41.5046 49.78917)


In [236]:
# Narrow meteorites down to only those that where witnessed
gdf = gdf[gdf["fall"] == "Fell"]

### Create a Shapefile

In [237]:
# Create a Shapefile from the geo data frame
if not os.path.isdir("./shapefiles"):
    os.mkdir("./shapefiles")
gdf.to_file("./shapefiles/meteorites.shp")

/Users/fosse/Desktop/python/meteorite-geospatial-data-visualization/my-env/lib/python3.13/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [238]:
# Compress the Shapefile parts into a ZIP file, simultaneously removing them
with ZipFile("./shapefiles/meteorites.zip", "w") as zipf:
    for file in os.listdir(os.fsencode("./shapefiles/")):
      filename = os.fsdecode(file)
      if not filename.endswith(".zip"):
        zipf.write("./shapefiles/" + filename)
        os.remove("./shapefiles/" + filename)